### Goal
Implement the following linear regression and apply on blood to muscle samples.

### TODO
   1. simulated dataset
   2. 4 x regression models:<br/>
       2.1 model1 $Y_i,g,t = beta_g_t*Y_i,g,t=blood$<br/>
       2.2 model2 $Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero$<br/>
       2.3 model3 $Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_t_k * P_i_k)$<br/>
       2.4 model4 $Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_t_k * P_i_k) + sum_l (beta_t_l * Q_g_l)$, where g-gene, i-individual, t-tissue, k=0,n_comp_P-1, l=0,n_comp_Q-1<br/>
   3. Apply model 4 on the real data
### Conclusions

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, KernelPCA, IncrementalPCA
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LassoLarsCV, LinearRegression, RidgeCV, SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

from scipy.stats import spearmanr, pearsonr
from scipy.stats import nbinom

np.random.seed(1)

### 1. Simulate data

In [3]:
# fix values
n_genes = 10
n_indiv = 30
n_indiv_effect = 2
n_gene_effect = 2
pca_type = PCA

# input
y_blood = np.random.uniform(low=-6, high=6, size=(n_indiv,n_genes))

# PCA - individual effect
s = StandardScaler()
ys_blood = s.fit_transform(y_blood)
pca = pca_type(n_components=n_indiv_effect)
# indiv effect
P = pca.fit_transform(ys_blood)

# PCA - gene effect
s = StandardScaler()
ys_blood = s.fit_transform(y_blood.T)
pca = pca_type(n_components=n_gene_effect)
# gene effect
Q = pca.fit_transform(ys_blood)

print(f'P_shape = {P.shape}, Q_shape={Q.shape}, input_shape={y_blood.shape}')

# values to approximate
beta_zero = 0.001
alphas = np.random.uniform(-1,1,size=n_indiv_effect)
betas = np.random.uniform(-1,1,size=n_gene_effect)
tetas = np.random.uniform(-1,1,size=n_genes)

tetas_matrix = np.zeros((n_genes,n_genes))
tetas_matrix= np.diag(tetas)

# final output Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_t_k * P_i_k) + sum_l (beta_t_l * Q_g_l)
y_out = y_blood@tetas_matrix  + beta_zero + np.tile((P@alphas).reshape(n_indiv,1),(1,n_genes)) + np.tile((Q@betas).T.reshape(1,n_genes),(n_indiv,1))
print(f'out_shape={y_out.shape}')

P_shape = (30, 2), Q_shape=(10, 2), input_shape=(30, 10)
out_shape=(30, 10)


### 2. Regression models
#### 2.1. Compute : $Y_i,g,t=beta_g_t*Y_i,g,t=blood$, i=individual, g=gene, t=tissue. 

In [4]:
y_out = y_blood@tetas_matrix
lm_array = []

print("Fitted Param:")
for gene in range(n_genes):
    X = y_blood[:,gene].reshape(-1, 1)
    y = y_out[:,gene].reshape(-1, 1)
    lm = LinearRegression(fit_intercept=False).fit(X,y)
    lm_array.append(lm)
    print(lm.coef_)
print("\nComputed Param:")
print(tetas)

# predict
y_pred = np.zeros(y_out.shape)
for i,lm in enumerate(lm_array):
    y_pred[:,i] = lm.predict(y_blood[:,i].reshape(-1, 1)).squeeze()

print("\nObs")
print(y_out[:3,:3])
print("\nPred")
print(y_pred[:3,:3])
print("\nObs-Pred=")
print(np.sum(y_out-y_pred))

Fitted Param:
[[-0.67805713]]
[[-0.06623995]]
[[-0.3096559]]
[[-0.54992008]]
[[0.18502374]]
[[-0.37546032]]
[[0.83261111]]
[[0.81927105]]
[[-0.48576341]]
[[-0.7782174]]

Computed Param:
[-0.67805713 -0.06623995 -0.3096559  -0.54992008  0.18502374 -0.37546032
  0.83261111  0.81927105 -0.48576341 -0.7782174 ]

Obs
[[ 0.67516585 -0.17513141  1.85751038]
 [ 0.65748882 -0.14722718  1.09821725]
 [-2.44706398 -0.37221151  0.69329164]]

Pred
[[ 0.67516585 -0.17513141  1.85751038]
 [ 0.65748882 -0.14722718  1.09821725]
 [-2.44706398 -0.37221151  0.69329164]]

Obs-Pred=
-8.760353553682876e-15


#### 2.2. Compute: $Y_i,g,t=beta_g_t*Y_i,g,t=blood + beta_zero$ and optimize the computation using parallelization

In [5]:
from joblib import Parallel, delayed

def train_model(X_, y_, fit_intercept=True):
    m = LinearRegression(fit_intercept=fit_intercept)
    return m.fit(X_, y_)

def model2(X,y,size_genes, fit_intercept):
    lm_array = []
    lm_array = Parallel(n_jobs=4)(delayed(train_model)(X[:,gene].reshape(-1,1), y[:,gene].reshape(-1,1), fit_intercept) for gene in range(size_genes))
    return lm_array

def predict_model2(model, X):
    y_first = model[0].predict(X[:,0].reshape(-1,1))
    y_pred = np.zeros((y_first.shape[0], len(model)))
    y_pred[:,0] = y_first.squeeze()
    for i in range(1,len(model)):
        y_pred[:,i] = (model[i].predict(X[:,i].reshape(-1,1))).squeeze()
    return y_pred

def coef_model2(model):
    print("\nFitted Param:")
    print("Tetas: ")
    for lm in model: print(f'coef={lm.coef_};intercept={lm.intercept_}')
    print("")

In [7]:
# true output
y_out = y_blood@tetas_matrix + beta_zero

# fit  
model_obj = model2(y_blood,y_out,n_genes,fit_intercept=True)

# predict
y_pred = predict_model2(model_obj,y_blood)
                   
# print coeficients
coef_model2(model_obj)

print("\nComputed Param:")
print(tetas)
print(beta_zero)

print("\nObs")
print(y_out[:3,:3])
print("\nPred")
print(y_pred[:3,:3])
print("\nObs-Pred=")
print(np.sum(y_out-y_pred))


Fitted Param:
Tetas: 
coef=[[-0.67805713]];intercept=[0.001]
coef=[[-0.06623995]];intercept=[0.001]
coef=[[-0.3096559]];intercept=[0.001]
coef=[[-0.54992008]];intercept=[0.001]
coef=[[0.18502374]];intercept=[0.001]
coef=[[-0.37546032]];intercept=[0.001]
coef=[[0.83261111]];intercept=[0.001]
coef=[[0.81927105]];intercept=[0.001]
coef=[[-0.48576341]];intercept=[0.001]
coef=[[-0.7782174]];intercept=[0.001]


Computed Param:
[-0.67805713 -0.06623995 -0.3096559  -0.54992008  0.18502374 -0.37546032
  0.83261111  0.81927105 -0.48576341 -0.7782174 ]
0.001

Obs
[[ 0.67616585 -0.17413141  1.85851038]
 [ 0.65848882 -0.14622718  1.09921725]
 [-2.44606398 -0.37121151  0.69429164]]

Pred
[[ 0.67616585 -0.17413141  1.85851038]
 [ 0.65848882 -0.14622718  1.09921725]
 [-2.44606398 -0.37121151  0.69429164]]

Obs-Pred=
7.27456289650874e-15


#### 2.3. Compute: $Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_k_t * P_i_k)$
For this model we need to estimate:
- the tissue specific intercept (beta_zero) 
- the coeficients (alphas) for the individual effect (P matrix)
- the weights for each gene-tissue (beta_g_t)

In [22]:
# true output
y_out = y_blood@tetas_matrix  + beta_zero + np.tile((P@alphas).reshape(n_indiv,1),(1,n_genes))

def model3(X,X_P,y,size_genes, fit_intercept):
    lm_array = []
    lm_array = Parallel(n_jobs=4)(delayed(train_model)(np.concatenate((X_P, X[:,gene].reshape(-1,1)), axis=1), y[:,gene].reshape(-1,1), fit_intercept) for gene in range(size_genes))
    return lm_array

def predict_model3(model, X, X_P):
    y_first = model[0].predict(np.concatenate((X_P, X[:,0].reshape(-1,1)), axis=1))
    y_pred = np.zeros((y_first.shape[0], len(model)))
    y_pred[:,0] = y_first.squeeze()
    for i in range(1,len(model)):
        y_pred[:,i] = (model[i].predict(np.concatenate((X_P, X[:,i].reshape(-1,1)), axis=1))).squeeze()
    return y_pred

def coef_model3(model):
    print("\nFitted Param:")
    print("Tetas: ")
    for lm in model: print(f'coef={lm.coef_};intercept={lm.intercept_}')
    print("") 
    
# fit
model = model3(y_blood,P,y_out,n_genes, fit_intercept = True)
# predict
y_pred = predict_model3(model,y_blood,P)
# coef
coef_model3(model)

print("\nComputed Param:")
print("Tetas ",tetas)
print("Beta_zero ", beta_zero)
print("Alphas ",alphas)

print("\nObs")
print(y_out[:3,:3])
print("\nPred")
print(y_pred[:3,:3])



Fitted Param:
Tetas: 
coef=[[ 0.6237174   0.74992329 -0.67805713]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.06623995]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.3096559 ]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.54992008]];intercept=[0.001]
coef=[[0.6237174  0.74992329 0.18502374]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.37546032]];intercept=[0.001]
coef=[[0.6237174  0.74992329 0.83261111]];intercept=[0.001]
coef=[[0.6237174  0.74992329 0.81927105]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.48576341]];intercept=[0.001]
coef=[[ 0.6237174   0.74992329 -0.7782174 ]];intercept=[0.001]


Computed Param:
Tetas  [-0.67805713 -0.06623995 -0.3096559  -0.54992008  0.18502374 -0.37546032
  0.83261111  0.81927105 -0.48576341 -0.7782174 ]
Beta_zero  0.001
Alphas  [0.6237174  0.74992329]

Obs
[[-0.31079402 -1.16109128  0.87155051]
 [-0.22202064 -1.02673664  0.21870778]
 [ 0.46493827  2.53979074  3.60529389]]

Pred
[[-0.31079402 -1.16109128  0.8

#### 2.4. Compute $Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_t_k * P_i_k) + sum_l (beta_t_l * Q_g_l)$


In [40]:
# final output Y_i,g,t = beta_g_t*Y_i,g,t=blood + beta_zero + sum_k (alpha_t_k * P_i_k) + sum_l (beta_t_l * Q_g_l)
y_out = y_blood@tetas_matrix  + beta_zero + np.tile((P@alphas).reshape(n_indiv,1),(1,n_genes)) + np.tile((Q@betas).reshape(1,n_genes),(n_indiv,1))

def model4(X,X_P,X_Q,y,size_genes, size_indiv, fit_intercept):
    Q_block = np.tile(X_Q,(size_indiv,1))
    
    P_block = np.repeat(P, repeats=size_genes, axis=0)
    
    X_block = np.diag(X[0,:])
    for row in range(1, X.shape[0]): X_block = np.vstack([X_block,np.diag(X[row,:])])
        
    print(Q_block.shape,P_block.shape,X_block.shape, y.flatten().shape)
    
    model = LinearRegression(fit_intercept=fit_intercept).fit(np.hstack([Q_block,P_block,X_block]),y.flatten())
    
    return model


def predict_model4(model, X, X_P, X_Q, size_indiv, size_genes):
    
    Q_block = np.tile(X_Q,(size_indiv,1))
    
    P_block = np.repeat(P, repeats=size_genes, axis=0)
    
    X_block = np.diag(X[0,:])
    for row in range(1, X.shape[0]): X_block = np.vstack([X_block,np.diag(X[row,:])])
    
    return model.predict(np.hstack([Q_block,P_block,X_block]))

def coef_model4(model):
    print("\nFitted Param:")
    print(f'Betas:{model.coef_[:2]}')
    print(f'Alphas:{model.coef_[2:4]}')
    print(f'Tetas:{model.coef_[4:]}')
    print(f'Beta_zero:{model.intercept_}')
    
# fit
model = model4(y_blood,P,Q,y_out,n_genes,n_indiv, fit_intercept = True)
# predict
y_pred = predict_model4(model,y_blood,P,Q, n_indiv, n_genes)
# coef
coef_model4(model)

print("\nComputed Param:")
print("Tetas ",tetas)
print("Beta_zero ", beta_zero)
print("Alphas ",alphas)
print("Betas ",betas)


print("\nObs")
print(y_out[:3,:3])
print("\nPred")
print(y_pred.reshape(n_indiv, n_genes)[:3,:3])

(300, 2) (300, 2) (300, 10) (300,)

Fitted Param:
Betas:[0.3768265  0.13898883]
Alphas:[0.6237174  0.74992329]
Tetas:[-0.67805713 -0.06623995 -0.3096559  -0.54992008  0.18502374 -0.37546032
  0.83261111  0.81927105 -0.48576341 -0.7782174 ]
Beta_zero:0.0010000000000001674

Computed Param:
Tetas  [-0.67805713 -0.06623995 -0.3096559  -0.54992008  0.18502374 -0.37546032
  0.83261111  0.81927105 -0.48576341 -0.7782174 ]
Beta_zero  0.001
Alphas  [0.6237174  0.74992329]
Betas  [0.3768265  0.13898883]

Obs
[[ 0.12013442 -0.58621628  0.09686965]
 [ 0.2089078  -0.45186164 -0.55597308]
 [ 0.89586671  3.11466574  2.83061302]]

Pred
[[ 0.12013442 -0.58621628  0.09686965]
 [ 0.2089078  -0.45186164 -0.55597308]
 [ 0.89586671  3.11466574  2.83061302]]
